# Prepare data for data publishing to Zenodo for Wills 2025 SaFT paper

- Simulations: ERA5 simulation, `/g/data/ik11/outputs/access-om2-025/025deg_era5_iaf/`
- Time period: 1980-2021 = 42 years of data

In [ ]:
#Load required packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from tqdm import tqdm
import glob
import sys, os

In [ ]:
# ERA5 run:
# -----------------
base = '/g/data/ik11/outputs/access-om2-025/025deg_era5_iaf/'
outname = 'ACCESS-OM2-025_ERA5_'

# 1980-2021:
years = np.arange(1980,2022,1)
outputs = np.arange(0, 42,1)

In [ ]:
ocean_vars = {'temp':'ocean/ocean-3d-temp-1-monthly-mean-ym_%04d_01.nc',
             'salt':'ocean/ocean-3d-salt-1-monthly-mean-ym_%04d_01.nc',
             'net_sfc_heating':'ocean/ocean-2d-net_sfc_heating-1-monthly-mean-ym_%04d_01.nc',
             'mld':'ocean/ocean-2d-mld-1-monthly-mean-ym_%04d_01.nc'}
ice_vars  = ['aice_m','tarea']

output_dir = '/g/data/e14/rmh561/Hobbs_2025_SaFT_project/data_publishing/'

region = [-360, 360, -90, -45]
depths = [0, 20]
rho0 = 1035.
Cp = 3992.10322329649

In [ ]:
# Grid variables:
dsgrid = xr.open_dataset(base + 'output%03d/ocean/' % outputs[0] + 'ocean-2d-geolon_t.nc').sel(xt_ocean=slice(region[0],region[1]),yt_ocean=slice(region[2],region[3])).isel(time=0,drop=True).load()
dsgrid['geolat_t'] = xr.open_dataset(base + 'output%03d/ocean/' % outputs[0] + 'ocean-2d-geolat_t.nc')['geolat_t'].sel(xt_ocean=slice(region[0],region[1]),yt_ocean=slice(region[2],region[3])).load()

# st_edges_ocean:
dsgrid['st_edges_ocean'] = xr.open_dataset(base + 'output%03d/ocean/' % outputs[0] + 'ocean-3d-temp-1-monthly-mean-ym_%04d_01.nc' % years[0])['st_edges_ocean']

In [ ]:
for i, output in tqdm(enumerate(outputs)):
    year = years[i]
    
    ##### OCEAN VARS:
    
    # Load vars into a dataset:
    vari = list(ocean_vars.keys())[0]
    file = ocean_vars[vari]
    ds = xr.open_dataset(base + 'output%03d/' % output + file % year)[vari].to_dataset()
    for var, file in ocean_vars.items():
        if var != vari: 
            ds[var] = xr.open_dataset(base + 'output%03d/' % output + file % year)[var]
    
    # Subselect region:
    ds = ds.sel(xt_ocean=slice(region[0],region[1]),yt_ocean=slice(region[2],region[3])).isel(st_ocean=slice(depths[0],depths[1]+1))
    
    # Load:
    ds.load()
    
    # Add coordinates:
    ds = ds.assign_coords({'geolon_t':dsgrid['geolon_t'],'geolat_t':dsgrid['geolat_t'],'st_edges_ocean':dsgrid['st_edges_ocean'].isel(st_edges_ocean=slice(0,len(ds.st_ocean)+1))})
    
    # Save to netcdf:
    ds.to_netcdf(output_dir + outname + 'ocean_vars_' + str(year) + '.nc')
    
    ##### ICE VARS:
    
    # Load vars into a dataset:
    ds_ice = xr.open_mfdataset([base + 'output%03d/ice/OUTPUT/iceh.%04d-' % (output,year) + '%02d.nc' % (x+1) for x in range(12)]
                           ,concat_dim='time',coords=['time','TLON','TLAT'],combine='nested',compat='override',parallel=True)[ice_vars]
                           #data_vars=ice_vars,concat_dim='time',combine='nested', 
                           #coords=['time','TLON','TLAT','tmask','tarea'],compat='override',parallel=True,drop_variables=['NCAT'])
    
    # Subselect region:
    ds_ice = ds_ice.sel(nj=slice(0,len(ds.yt_ocean)))

    # Get rid of time in tarea:
    ds_ice['tarea'] = ds_ice.tarea.isel(time=0,drop=True)
    
    # Load:
    ds_ice.load()
    
    # Save to netcdf:
    ds_ice.to_netcdf(output_dir + outname + 'ice_vars_' + str(year) + '.nc')